In [1]:
from flask import Flask, jsonify, abort, request
import re
import threading

app = Flask(__name__)

# Function to check for HTML tags
def contains_html_tags(text):
    html_tag_pattern = re.compile(r'<[^>]+>')
    return bool(html_tag_pattern.search(text))

@app.route('/api/greet', methods=['GET'])
def greet():
    name = request.args.get('name')
    if not name:
        abort(400, description="Name parameter is required.")
    if contains_html_tags(name):
        abort(400, description="HTML content is not allowed in the name parameter.")
    return jsonify(message=f"Hello, {name}!")

@app.errorhandler(400)
def handle_bad_request(e):
    response = jsonify(error=str(e))
    response.status_code = 400
    return response

@app.errorhandler(404)
def handle_not_found(e):
    response = jsonify(error="Not Found")
    response.status_code = 404
    return response

# To get it to work in Colab, use this line instead of the usual app.run
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [2]:
import unittest
from flask.testing import FlaskClient

class FlaskTestClient(FlaskClient):
    def open(self, *args, **kwargs):
        kwargs.setdefault('follow_redirects', True)
        return super().open(*args, **kwargs)

app.test_client_class = FlaskTestClient

class TestFlaskAPI(unittest.TestCase):
    def setUp(self):
        self.app = app.test_client()
        self.app.testing = True

    def test_basic_greeting(self):
        response = self.app.get('/api/greet?name=John')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, John!"})

    def test_url_encoding(self):
        response = self.app.get('/api/greet?name=John%20Doe')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, John Doe!"})

    def test_special_characters(self):
        response = self.app.get('/api/greet?name=@!$*')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, @!$*!"})

    def test_empty_name(self):
        response = self.app.get('/api/greet')
        self.assertEqual(response.status_code, 400)
        self.assertEqual(response.json, {"error": "400 Bad Request: Name parameter is required."})

    def test_numeric_name(self):
        response = self.app.get('/api/greet?name=12345')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, 12345!"})

    def test_long_name(self):
        long_name = "a" * 1000
        response = self.app.get(f'/api/greet?name={long_name}')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": f"Hello, {long_name}!"})

    def test_html_injection(self):
        html_name = "<script>alert('test')</script>"
        response = self.app.get(f'/api/greet?name={html_name}')
        # Note that this was added to double check. THis is why you'll see the error in the output, even if the test passed.
        print(response.json)
        self.assertEqual(response.status_code, 400)
        self.assertEqual(response.json, {"error": "400 Bad Request: HTML content is not allowed in the name parameter."})

    def test_json_response_format(self):
        response = self.app.get('/api/greet?name=John')
        self.assertEqual(response.headers['Content-Type'], 'application/json')
        self.assertTrue(response.json)

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_basic_greeting (__main__.TestFlaskAPI) ... ok
test_empty_name (__main__.TestFlaskAPI) ... ok
test_html_injection (__main__.TestFlaskAPI) ... ok
test_json_response_format (__main__.TestFlaskAPI) ... ok
test_long_name (__main__.TestFlaskAPI) ... ok
test_numeric_name (__main__.TestFlaskAPI) ... ok
test_special_characters (__main__.TestFlaskAPI) ... ok
test_url_encoding (__main__.TestFlaskAPI) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.128s

OK


{'error': '400 Bad Request: HTML content is not allowed in the name parameter.'}
